In [ ]:
#迁移学习resnet50

import cv2
import glob
import numpy as np
import os
import tensorflow as tf

from tensorflow.contrib.slim import nets

slim = tf.contrib.slim

flags = tf.flags

FLAGS = flags.FLAGS

images_dir = './manual/review_img1'

sku_label={}
file=open('./manual/review_data1_label.txt','r',encoding='utf8')
for line in file:
    sku=line.split('\t')[0]
    label=line.split('\t')[1]
    sku_label[sku]=int(label)
file.close()

def get_test_data(images_dir='./predict'):
    images = []
    labels = []
    image_files = np.array(glob.glob(os.path.join(images_dir, '*.jpg')))
    for image_path in image_files:
        sku = image_path.split('/')[-1].split('.')[0].split('_')[1]
        label = image_path.split('/')[-1].split('.')[0].split('_')[0]
        image = cv2.imread(image_path)
        try:
            image = cv2.resize(image, (224, 224))
        except:
            image=np.zeros((224,224,3))
        images.append(image)
        labels.append(int(label))
    images = np.array(images)
    labels = np.array(labels)
    return images, labels


def get_next_batch(batch_size=128, images_dir='../../pic_down_pre/small_pic3'):
    if not os.path.exists(images_dir):
        raise ValueError('`images_dir` is not exist.')
    images = []
    labels = []
    image_files = np.array(glob.glob(os.path.join(images_dir, '*.jpg')))
    batch_size = min(batch_size, len(image_files))
    selected_indices = np.random.choice(len(image_files), batch_size)
    selected_images_path = image_files[selected_indices]
    for image_path in selected_images_path:
        sku = image_path.split('/')[-1].split('.')[0]
        if sku in sku_label.keys():
            image = cv2.imread(image_path)
            try:
                image = cv2.resize(image, (224, 224))
            except:
                image=np.zeros((224,224,3))
            label = sku_label[sku]
            images.append(image)
            labels.append(int(label))
    images = np.array(images)
    labels = np.array(labels)
    return images, labels


def main(_):
    batch_size = 128
    num_classes = 2
    num_steps = 500
    checkpoint_path = './resnet_v1_50.ckpt'
    keep_prob=tf.placeholder(tf.float32, name='keep_prob')
    if checkpoint_path is None:
        raise ValueError('`checkpoint_path` must be specified.')
    model_save_dir = './output_base/1'
    if model_save_dir is None:
        model_save_dir = './training'
    if not os.path.exists(model_save_dir):
        os.mkdir(model_save_dir)
    model_save_path = os.path.join(model_save_dir, 'model.ckpt')
    # Generate training data
    

    inputs = tf.placeholder(tf.float32, shape=[None, 224, 224, 3], name='inputs')
    labels = tf.placeholder(tf.int32, shape=[None], name='labels')
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    with slim.arg_scope(nets.resnet_v1.resnet_arg_scope()):
        net, endpoints = nets.resnet_v1.resnet_v1_50(inputs, num_classes=None,
                                                     is_training=is_training)
        
    with tf.variable_scope('Logits'):
        net = tf.squeeze(net, axis=[1, 2])
        net = slim.dropout(net, keep_prob=keep_prob, scope='scope')
        logits = slim.fully_connected(net, num_outputs=num_classes,
                                      activation_fn=None, scope='fc')
        
    checkpoint_exclude_scopes = 'Logits'
    exclusions = None
    if checkpoint_exclude_scopes:
        exclusions = [
            scope.strip() for scope in checkpoint_exclude_scopes.split(',')]
    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
        if not excluded:
            variables_to_restore.append(var)
            
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits)
    loss = tf.reduce_mean(losses)
    
    logits = tf.nn.softmax(logits)
    classes = tf.argmax(logits, axis=1, name='classes')
    accuracy = tf.reduce_mean(tf.cast(
        tf.equal(tf.cast(classes, dtype=tf.int32), labels), dtype=tf.float32))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
    train_step = optimizer.minimize(loss)
    
    init = tf.global_variables_initializer()
    
    saver_restore = tf.train.Saver(var_list=variables_to_restore)
    saver = tf.train.Saver(tf.global_variables(),max_to_keep=2)
    
#    config = tf.ConfigProto(allow_soft_placement = True) 
#    config.gpu_options.per_process_gpu_memory_fraction = 0.95
#    with tf.Session(config=config) as sess:
    test_data,test_label=get_test_data('./predict')
    with tf.Session() as sess:
        sess.run(init)
        
        # Load the pretrained checkpoint file xxx.ckpt
        saver_restore.restore(sess, checkpoint_path)
        loss_min=0.002
        acc_min=0.9
        for i in range(num_steps):
            images, groundtruth_lists = get_next_batch(batch_size, images_dir)
            train_dict = {inputs: images, 
                          labels: groundtruth_lists,
                          keep_prob:0.5,
                          is_training: True}
            test_dict = {inputs: test_data, 
                          labels: test_label,
                          keep_prob:1.0,
                          is_training: False}

            sess.run(train_step, feed_dict=train_dict)
            
            loss_, acc_ = sess.run([loss, accuracy], feed_dict=train_dict)
            
            train_text = 'Step: {}, Train Loss: {:.4f}, Accuracy: {:.4f}'.format(
                i+1, loss_, acc_)
            test_loss_, test_acc_ = sess.run([loss, accuracy], feed_dict=test_dict)
            test_text = 'Test Loss: {:.4f}, Accuracy: {:.4f}'.format(test_loss_, test_acc_)
            print(train_text,test_text)
            if i == num_steps -1 or test_acc_>acc_min:
                acc_min=test_acc_
                saver.save(sess, model_save_path+'_'+str(i))
                print('save mode to {}'.format(model_save_dir))
                

if __name__ == '__main__':
    tf.app.run()

INFO:tensorflow:Restoring parameters from ./resnet_v1_50.ckpt
Step: 1, Train Loss: 0.2089, Accuracy: 0.9364 Test Loss: 0.5191, Accuracy: 0.7497
Step: 2, Train Loss: 0.1563, Accuracy: 0.9537 Test Loss: 0.3859, Accuracy: 0.8626
Step: 3, Train Loss: 0.1249, Accuracy: 0.9630 Test Loss: 0.2999, Accuracy: 0.9197
save mode to ./output_base/1
Step: 4, Train Loss: 0.0782, Accuracy: 0.9732 Test Loss: 0.2464, Accuracy: 0.9429
save mode to ./output_base/1
Step: 5, Train Loss: 0.0821, Accuracy: 0.9730 Test Loss: 0.2150, Accuracy: 0.9510
save mode to ./output_base/1
Step: 6, Train Loss: 0.1148, Accuracy: 0.9554 Test Loss: 0.1936, Accuracy: 0.9537
save mode to ./output_base/1
Step: 7, Train Loss: 0.0744, Accuracy: 0.9630 Test Loss: 0.1773, Accuracy: 0.9592
save mode to ./output_base/1
Step: 8, Train Loss: 0.1229, Accuracy: 0.9633 Test Loss: 0.1588, Accuracy: 0.9578
Step: 9, Train Loss: 0.0820, Accuracy: 0.9817 Test Loss: 0.1483, Accuracy: 0.9633
save mode to ./output_base/1
Step: 10, Train Loss: 0.08

In [ ]:
# 使用多次迭代的结果来训练base模型

import cv2
import glob
import numpy as np
import os
import tensorflow as tf

from tensorflow.contrib.slim import nets


slim = tf.contrib.slim

flags = tf.flags


FLAGS = flags.FLAGS

images_dir = './manual/review_img'

sku_label={}
file=open('./manual/review_data_label_1_2_3.txt','r',encoding='utf8')
for line in file:
    sku=line.split('\t')[0]
    label=line.split('\t')[1]
    sku_label[sku]=int(label)
file.close()

for i in range(18,1009):
    sku_label[str(i)]=1
def get_test_data(images_dir='./predict'):
    images = []
    labels = []
    image_files = np.array(glob.glob(os.path.join(images_dir, '*.jpg')))
    for image_path in image_files:
        sku = image_path.split('/')[-1].split('.')[0].split('_')[1]
        label = image_path.split('/')[-1].split('.')[0].split('_')[0]
        image = cv2.imread(image_path)
        try:
            image = cv2.resize(image, (224, 224))
        except:
            image=np.zeros((224,224,3))
        images.append(image)
        labels.append(int(label))
    images = np.array(images)
    labels = np.array(labels)
    return images, labels


def get_next_batch(batch_size=128, images_dir='../../pic_down_pre/small_pic3'):
    if not os.path.exists(images_dir):
        raise ValueError('`images_dir` is not exist.')
    images = []
    labels = []
    image_files = np.array(glob.glob(os.path.join(images_dir, '*.jpg')))
    batch_size = min(batch_size, len(image_files))
    selected_indices = np.random.choice(len(image_files), batch_size)
    selected_images_path = image_files[selected_indices]
    for image_path in selected_images_path:
        sku = image_path.split('/')[-1].split('.')[0]
        if sku in sku_label.keys():
            image = cv2.imread(image_path)
            try:
                image = cv2.resize(image, (224, 224))
            except:
                image=np.zeros((224,224,3))
            label = sku_label[sku]
            images.append(image)
            labels.append(int(label))
    images = np.array(images)
    labels = np.array(labels)
    return images, labels


def main(_):
    batch_size = 128
    num_classes = 2
    num_steps = 500
    checkpoint_path = './resnet_v1_50.ckpt'
    keep_prob=tf.placeholder(tf.float32, name='keep_prob')
    if checkpoint_path is None:
        raise ValueError('`checkpoint_path` must be specified.')
    model_save_dir = './output_base/1_2_3'
    if model_save_dir is None:
        model_save_dir = './training'
    if not os.path.exists(model_save_dir):
        os.mkdir(model_save_dir)
    model_save_path = os.path.join(model_save_dir, 'model.ckpt')
    # Generate training data
    

    inputs = tf.placeholder(tf.float32, shape=[None, 224, 224, 3], name='inputs')
    labels = tf.placeholder(tf.int32, shape=[None], name='labels')
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    with slim.arg_scope(nets.resnet_v1.resnet_arg_scope()):
        net, endpoints = nets.resnet_v1.resnet_v1_50(inputs, num_classes=None,
                                                     is_training=is_training)
        
    with tf.variable_scope('Logits'):
        net = tf.squeeze(net, axis=[1, 2])
        net = slim.dropout(net, keep_prob=keep_prob, scope='scope')
        logits = slim.fully_connected(net, num_outputs=num_classes,
                                      activation_fn=None, scope='fc')
        
    checkpoint_exclude_scopes = 'Logits'
    exclusions = None
    if checkpoint_exclude_scopes:
        exclusions = [
            scope.strip() for scope in checkpoint_exclude_scopes.split(',')]
    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
        if not excluded:
            variables_to_restore.append(var)
            
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits)
    loss = tf.reduce_mean(losses)
    
    logits = tf.nn.softmax(logits)
    classes = tf.argmax(logits, axis=1, name='classes')
    accuracy = tf.reduce_mean(tf.cast(
        tf.equal(tf.cast(classes, dtype=tf.int32), labels), dtype=tf.float32))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
    train_step = optimizer.minimize(loss)
    
    init = tf.global_variables_initializer()
    
    saver_restore = tf.train.Saver(var_list=variables_to_restore)
    saver = tf.train.Saver(tf.global_variables(),max_to_keep=5)
    
#    config = tf.ConfigProto(allow_soft_placement = True) 
#    config.gpu_options.per_process_gpu_memory_fraction = 0.95
#    with tf.Session(config=config) as sess:
    test_data,test_label=get_test_data('./predict')
    with tf.Session() as sess:
        sess.run(init)
        
        # Load the pretrained checkpoint file xxx.ckpt
        saver_restore.restore(sess, checkpoint_path)
        loss_min=0.002
        acc_min=0.9
        for i in range(num_steps):
            images, groundtruth_lists = get_next_batch(batch_size, images_dir)
            train_dict = {inputs: images, 
                          labels: groundtruth_lists,
                          keep_prob:0.4,
                          is_training: True}
            test_dict = {inputs: test_data, 
                          labels: test_label,
                          keep_prob:1.0,
                          is_training: False}

            sess.run(train_step, feed_dict=train_dict)
            
            loss_, acc_ = sess.run([loss, accuracy], feed_dict=train_dict)
            
            train_text = 'Step: {}, Train Loss: {:.4f}, Accuracy: {:.4f}'.format(
                i+1, loss_, acc_)
            test_loss_, test_acc_ = sess.run([loss, accuracy], feed_dict=test_dict)
            test_text = 'Test Loss: {:.4f}, Accuracy: {:.4f}'.format(test_loss_, test_acc_)
            print(train_text,test_text)
            if i == num_steps -1 or test_acc_>acc_min:
                acc_min=test_acc_
                saver.save(sess, model_save_path+'_'+str(i))
                print('save mode to {}'.format(model_save_dir))
                

if __name__ == '__main__':
    tf.app.run()

INFO:tensorflow:Restoring parameters from ./resnet_v1_50.ckpt
Step: 1, Train Loss: 0.3170, Accuracy: 0.8714 Test Loss: 0.6090, Accuracy: 0.6707
Step: 2, Train Loss: 0.3009, Accuracy: 0.8689 Test Loss: 0.5117, Accuracy: 0.7306
Step: 3, Train Loss: 0.2912, Accuracy: 0.8592 Test Loss: 0.4646, Accuracy: 0.7578
Step: 4, Train Loss: 0.2482, Accuracy: 0.9024 Test Loss: 0.4086, Accuracy: 0.7959
Step: 5, Train Loss: 0.2299, Accuracy: 0.9385 Test Loss: 0.3653, Accuracy: 0.8259
Step: 6, Train Loss: 0.0890, Accuracy: 0.9726 Test Loss: 0.3459, Accuracy: 0.8354
Step: 7, Train Loss: 0.1378, Accuracy: 0.9474 Test Loss: 0.3463, Accuracy: 0.8313
Step: 8, Train Loss: 0.0867, Accuracy: 0.9452 Test Loss: 0.3533, Accuracy: 0.8286
Step: 9, Train Loss: 0.1784, Accuracy: 0.9429 Test Loss: 0.3566, Accuracy: 0.8340
Step: 10, Train Loss: 0.0932, Accuracy: 0.9846 Test Loss: 0.3656, Accuracy: 0.8354
Step: 11, Train Loss: 0.0898, Accuracy: 0.9615 Test Loss: 0.3700, Accuracy: 0.8422
Step: 12, Train Loss: 0.1791, Accu